This practical is loosely based on the first Trident tutorial which can be found [here](https://github.com/mahmoodlab/TRIDENT/blob/main/tutorials/1-Step-by-Step-Patch-Feature-Extraction-with-Trident.ipynb)

# Practical 1: Whole slide images preprocessing
Whole slide images (WSIs) are challenging to process due to their very high resolution (generally 100,000 x 100,000 px or more) and various artifacts (different staining methods, noisy background, etc.). The usual pipeline to extract a computer-friendly vector representation of a WSI is as follows:
1. Segmentation: separating the tissue from the background.
2. Coordinate extraction: cutting the segmented WSI into smaller tiles, usually 512 x 512 px or 256 x 256 px at 20x magnification.
3. Tile embedding: generating a vector representation of each tile
4. Slide embedding: based on its tile embeddings, generating a vector representation of each WSI.
   
[Trident](https://github.com/mahmoodlab/TRIDENT/tree/main) is a python library which provides state of the art (SOTA) models for step 1, 3, and 4. Each step of the pipeline can be done through a single command line. For the purpose of this practical, all models used will be available offline. If you want to use them on your own after the pratical, you need to create an account on hugging face and request access to them ([here](https://huggingface.co/MahmoodLab/conchv1_5) for CONCH v 1.5, [there](https://huggingface.co/MahmoodLab/abmil.base.conch_v15.pc108-24k) for Feather and [there](https://huggingface.co/MahmoodLab/TITAN) for Titan).
If you download Trident, it is also possible to do all the steps at once, on a batch of slides with a single command line. For example, to generate feather slide embeddings of the WSIs stored in `<your/wsi/dir>` and save them into `your/output/dir`, this would be:

`python run_batch_of_slides.py --task all --wsi_dir your/wsi/dir --job_dir your/output/dir --slide_encoder feather --mag 20 --patch_size 512`.

Here, `task --all` indicates that you want to perform step 1 to 4, `--slide_encoder feather` says that you will use the slide encoder feather, with a magnification of `--mag 20` and a tile size of `--patch_size 512`

Given that this pratical is intended as a demo of Trident, we will instead work step by step on a single, but challenging, WSI taken from TCGA-MESO (case ID [	
TCGA-MQ-A6BQ](https://portal.gdc.cancer.gov/cases/d3a6accc-5c45-4e70-baf5-2188af53e0db), Epitehlioid mesothelioma)


In [ ]:
from IPython.display import Image
data_dir = "/data/Training-MG/files/data/AI_praticals_2025/AI_pratical_1_trident"
slide_name = "TCGA-MQ-A6BQ-01Z-00-DX1.72BF21E4-17D6-436B-AAD9-7960342894F4"
Image(filename=f"{data_dir}/hest/thumbnails/{slide_name}.jpg")

⚠️ Because this pratical is only two hours long and some things are very long to compute on CPU, we replace them with placeholder methods below.
If you want to test the true methods after the pratical on a GPU, just comment the lines of code below and run your code as before.

In [ ]:
import sys
sys.path.append(f"{data_dir}/utils")
from monkey_patch import segment_tissue, extract_patch_features, extract_slide_features
from trident.wsi_objects.OpenSlideWSI import OpenSlideWSI

OpenSlideWSI.segment_tissue = segment_tissue
OpenSlideWSI.extract_patch_features = extract_patch_features
OpenSlideWSI.extract_slide_features = extract_slide_features

## 1. Segmentation
This step will separate the tissue from the background.
Note that if you use the Trident repo on your own later on, you can also do this step with a batch of WSIs with the following command line:
`python run_batch_of_slides.py --task seg --wsi_dir your/wsi/dir --job_dir your/output/dir --segmenter hest`
This will run the segmentation on the batch of WSIs using Hest segmenter (`--segmenter hest`). The flag `--remove-artifacts` can optionnally be added to remove artifacts.

First, let's load the slide

In [ ]:
from trident import load_wsi
slide = load_wsi(slide_path=f"{data_dir}/slide/{slide_name}.svs", lazy_init=False)

Now, let's load the segmentation model

In [ ]:
from trident.segmentation_models import segmentation_model_factory
segmentation_model = segmentation_model_factory(model_name="hest")

If you want to see what HEST looks like

In [ ]:
segmentation_model

We can now segment the slide with the loaded model, at magnification 20x, and store them in a `trident_processed` folder.

In [ ]:
job_dir = "./trident_processed"
slide.segment_tissue(
    segmentation_model=segmentation_model,
    target_mag=20,
    job_dir=job_dir,
    holes_are_tissue=True,
    device='cpu',
    num_workers=8,
    verbose=True
    )

Let's have a look at the segmentation

In [ ]:
Image(filename=f"{job_dir}/contours/{slide_name}.jpg")

What do you think of the quality of the segmentation? How can it be improved?

By default, the segmenter consider holes as tissue. What happens if we set it to False?

In [ ]:
import copy
job_dir_no_holes = "./trident_processed_no_holes"
slide_no_holes = copy.copy(slide)
slide_no_holes.segment_tissue([...])  # Complete this code to rerun the segmentation and remove tiles containing holes

In [ ]:
Image(filename=f"{job_dir_no_holes}/contours/{slide_name}.jpg")

We can try to use an additional artifact remover to get rid of the tiles with penmarks.
This model is named `grandqc_artifact`. It can be loaded in the same way as the segmentation model

In [ ]:
artifact_remover_model = segmentation_model_factory(model_name=[...], remove_penmarks_only=True) # Complete this code to load grandqc_artifact

If you want to see what grandQC_artifact looks like

In [ ]:
artifact_remover_model

We now need to do a second pass of segment_tissue using the `artifact_remover_model`

In [ ]:
job_dir_no_pen = "./trident_processed_no_pen"
slide_no_pen = copy.copy(slide)
slide_no_pen.segment_tissue([...]) # Complete this code to rerun the segmentation and remove tiles containing penmarks

Let's have a look at the artifact removal. Is this better than the initial result?

In [ ]:
Image(filename=f"{job_dir_no_pen}/contours/{slide_name}.jpg")

## 2. Tiling
This step will split the WSI into smaller tiles.
Note that if you use the Trident repo on your own later on, you can also do this step with a batch of WSIs with the following command line:
`python run_batch_of_slides.py --task coords --wsi_dir your/wsi/dir --job_dir your/output/dir --mag 20 --patch_size 512 --overlap 0`
This will create tiles of size 512 x 512 px at 20x magnification, with 0px overlap.

In [ ]:
job_dir = "./trident_processed"
overlap = 0
coords_dir = f"{job_dir}/20x_512px_{overlap}px_overlap"
# We perform the tiling
coords_path = slide.extract_tissue_coords(
    target_mag=20,
    patch_size=512,
    min_tissue_proportion=0.,
    save_coords=coords_dir,
    overlap=overlap
)
# And visualise it
viz_coords_path = slide.visualize_coords(
    coords_path=coords_path,
    save_patch_viz=f"{coords_dir}/coord_visualization",
)
print(f"Tissue coordinates extracted and saved to {viz_coords_path}.")

Now, let's look at the tiling results. Did we get a lot of tiles using a shape of 512 x 512 px ? Does this take the previous segmentation into account?

In [ ]:
Image(viz_coords_path)

In [ ]:
import h5py
from lovely_numpy import lo

with h5py.File(f"./trident_processed/20x_512px_0px_overlap/patches/{slide_name}_patches.h5") as f: # Add the h5 filename you found above
    coords = f["coords"][:] # Extract the coordinates of the patches embeddings as a numpy array
print(lo(coords)) # Pretty display of the np array

Let's try this with some overlap.

In [ ]:
overlap = [...] # Change to the overlap value of your choice. It must be an positive integer lower than 512.
coords_dir_overlap = f"{job_dir}/20x_512px_{overlap}px_overlap"
# We perform the tiling
coords_path = slide.extract_tissue_coords(
    target_mag=20,
    patch_size=512,
    save_coords=coords_dir_overlap,
    overlap=overlap
)
# And visualise it
viz_coords_path = slide.visualize_coords(
    coords_path=coords_path,
    save_patch_viz=f"{coords_dir_overlap}/coord_visualization",
)
print(f"Tissue coordinates extracted and saved to {viz_coords_path}.")

What changes in the tiling?

In [ ]:
Image(viz_coords_path)

Now, complete both code cells below to compute the tiling for the segmentation version without holes and without penmarks.
We will use them in the next section.

In [ ]:
overlap = 0
coords_dir_no_holes = f"{job_dir_no_holes}/20x_512px_{overlap}px_overlap"
# Complete the code below to perform the tiling
coords_path_no_holes = slide_no_holes.extract_tissue_coords(
    [...]
)
# Complete the code below to visualise it
viz_coords_path_no_holes = slide_no_holes.visualize_coords(
    [...]
)
print(f"Tissue coordinates extracted and saved to {viz_coords_path_no_holes}.")

In [ ]:
overlap = 0
coords_dir_no_pen = f"{job_dir_no_pen}/20x_512px_{overlap}px_overlap"
# Complete the code below to perform the tiling
coords_path_no_pen = slide_no_pen.extract_tissue_coords(
    [...]
)
# Complete the code below to visualise it
viz_coords_path_no_pen = slide_no_pen.visualize_coords(
    [...]
)
print(f"Tissue coordinates extracted and saved to {viz_coords_path_no_pen}.")

## 3. Tile embedding
This step will transform the tiles into vector representations: the tile embeddings.
Note that if you use the Trident repo on your own later on, you can also do this step with a batch of WSIs with the following command line:
`python run_batch_of_slides.py --task feat --wsi_dir your/wsi/dir --job_dir your/output/dir --mag 20 --patch_size 512 --patch_encoder conch_v15`
This will create tile embeddings using the foundation model CONCH v 1.5.

In [ ]:
from trident.patch_encoder_models import encoder_factory
from trident.patch_encoder_models import encoder_registry as patch_encoder_registry
# We load CONCH v1.5
encoder = encoder_factory("conch_v15")
# We set it to evaluation mode, as we do not intend to fine tune it.
encoder.eval()

In [ ]:
# The results will be saved into ./trident_processed/20x_512px_0px_overlap/features_conch_v15
tiles_path = f"{coords_dir}/features_conch_v15"
coords_path = f"{coords_dir}/patches/{slide.name}_patches.h5"
slide.extract_patch_features(
        patch_encoder=encoder,
        coords_path=coords_path,
        save_features=tiles_path,
        batch_limit=16
    )

Now, let's look at the content of the folder. What is there?

In [ ]:
!ls ./trident_processed/20x_512px_0px_overlap/

Let's look inside the visualization folder listed above

In [2]:
!ls ./trident_processed_no_holes/20x_512px_0px_overlap/visualization

TCGA-MQ-A6BQ-01Z-00-DX1.72BF21E4-17D6-436B-AAD9-7960342894F4.jpg


Let's look inside the jpg file listed above

In [ ]:
Image(filename=f"./trident_processed_no_holes/20x_512px_0px_overlap/visualization/{slide_name}.jpg")

How many tiles were encoded for this slide? Does this match the number of tiles from the segmentation step? Why?

Let's repeat the process using the segmentation where we removed tiles with holes

In [ ]:
# The results will be saved into ./trident_processed_no_holes/20x_512px_0px_overlap/features_conch_v15
tiles_path_no_holes = [...]
coords_path_no_holes = [...]
# Fill in the code below to extract patch features using a segmentation where holes are removed
slide_no_holes.extract_patch_features([...])

Now, let's look at the content of the folder. What is there?

In [1]:
!ls ./trident_processed_no_holes/20x_512px_0px_overlap/

features_conch_v15  patches  slide_features_feather  visualisation


Let's look inside the visualization folder listed above

In [2]:
!ls ./trident_processed_no_holes/20x_512px_0px_overlap/visualization

TCGA-MQ-A6BQ-01Z-00-DX1.72BF21E4-17D6-436B-AAD9-7960342894F4.jpg


Let's look inside the jpg file listed above

In [ ]:
Image(filename=[...])

How many tiles were encoded for this slide? How many where removed compared to the classical segmentation? why?

Let's repeat the process using the segmentation where we removed tiles with penmarks

In [ ]:
# Fill in the code below to extract patch features using a segmentation where holes are removed
# The results will be saved into ./trident_processed_no_pen/20x_512px_0px_overlap/features_conch_v15
tiles_path_no_pen = [...]
coords_path_no_pen = [...]
slide_no_pen.extract_patch_features([...])

Fill in the code to retrieve the jpg file on the trident_processed_no_pen folder

In [ ]:
Image(filename=[...])

How many tiles were encoded for this slide? How many where removed compared to the classical segmentation? Are the tiles at the same coordinates?

## 4. Slide embedding
This step will transform the tiles into vector representations: the tile embeddings.
Note that if you use the Trident repo on your own later on, you can also do this step with a batch of WSIs with the following command line:
`python run_batch_of_slides.py --task feat --wsi_dir your/wsi/dir --job_dir your/output/dir --mag 20 --patch_size 512 --slide_encoder titan`
This will create slide embeddings using the foundation model Titan.

In [ ]:
from trident.slide_encoder_models.load import encoder_factory
slides_path = f"{coords_dir}/slide_features_feather"
slide_encoder = encoder_factory("feather")

In [ ]:
slide.extract_slide_features(
    patch_features_path=tiles_path,
    slide_encoder=slide_encoder,
    save_features=slides_path,
)

Now, let's look at the content of the folder `slides_path`. What is there?

In [ ]:
!ls ./trident_processed/20x_512px_0px_overlap/slide_features_feather

Let's look inside the h5 file listed above

In [ ]:
import h5py
from lovely_numpy import lo

with h5py.File([...]) as f: # Add the h5 filename you found above
    feats = f["features"][:] # Extract the slide embedding as a numpy array
    print(lo(feats))

What is the dimensionality of the obtained representation?

Now let's extract the slide features for tile embeddings without holes.

In [ ]:
# Here put the code to compute the slide embeddings using the segmentation where holes are removed
slides_path_no_holes = [...]
slide_no_holes.extract_slide_features([...])
!ls ./trident_processed_no_holes/20x_512px_0px_overlap/slide_features_feather
with h5py.File([...]) as f: # Add the h5 filename you found above
    feats_no_holes = f["features"][:] # Extract the slide embedding as a numpy array
print(lo(feats_no_holes))

Now let's extract the slide features for tile embeddings without penmarks.

In [ ]:
# Here put the code to compute the slide embeddings using the segmentation where penmarks are removed
slides_path_no_pen = [...]
slide_no_pen.extract_slide_features([...])
!ls ./trident_processed_no_pen/20x_512px_0px_overlap/slide_features_feather
with h5py.File([...]) as f: # Add the h5 filename you found above
    feats_no_pen = f["features"][:] # Extract the slide embedding as a numpy array
print(lo(feats_no_pen))

Now let's compare these representations.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
cosine_similarity(np.array([feats, feats_no_holes, feats_no_pen]))

How high is the similarity between the different slide embeddings? Did the different segmentation methods impact the slide representation?